In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain import hub

C:\Users\hp\anaconda\envs\Chameleon\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# --- 1. Configuration and LLM Initialization ---
load_dotenv()
llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.7)

In [3]:
# --- 2. Define the Structured Output ---
class HeadlineSet(BaseModel):
    """Data model for a set of generated headlines."""
    factual_headline: str = Field(description="A strictly factual, professional headline for the article.")
    creative_headline: str = Field(description="A creative, engaging, and slightly sensational headline for the article.")
    question_headline: str = Field(description="A headline phrased as an intriguing question to provoke curiosity.")
    target_audience: str = Field(description="The primary audience this article is likely for (e.g., 'Tech Enthusiasts', 'Political Analysts').")

In [4]:
# --- 3. Define Tools for the Agent ---

@tool
def get_article_summary(url: str) -> str:
    """
    Loads content from a URL, splits it into chunks, and returns a concise summary.
    Use this tool to understand the contents of a news article or webpage.
    """
    print(f"\n➡️ [Tool Running: get_article_summary] Loading content from {url}...")
    loader = WebBaseLoader(url)
    docs = loader.load()

    print("➡️ [Tool Running: get_article_summary] Splitting and summarizing...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
    split_docs = text_splitter.split_documents(docs)
    summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
    article_summary = summary_chain.invoke(split_docs)
    print("➡️ [Tool Running: get_article_summary] Summary complete.")
    return article_summary['output_text']

In [5]:
@tool
def generate_headlines(summary: str) -> dict:
    """
    Takes an article summary and generates a set of factual, creative, and question-based headlines.
    Use this tool after you have summarized an article.
    """
    print("➡️ [Tool Running: generate_headlines] Generating headlines...")
    pydantic_parser = PydanticOutputParser(pydantic_object=HeadlineSet)
    master_prompt = ChatPromptTemplate.from_template(
        """
        You are an expert copywriter. Analyze the following news article summary and generate a set of headlines based on the formatting instructions.

        Formatting Instructions:
        {format_instructions}

        Article Summary:
        ---
        {summary}
        ---
        """
    )
    structured_chain = master_prompt | llm | pydantic_parser
    headlines = structured_chain.invoke({
        "summary": summary,
        "format_instructions": pydantic_parser.get_format_instructions()
    })
    print("➡️ [Tool Running: generate_headlines] Headlines generated.")
    # Return the dictionary representation for clean output
    return headlines.model_dump()

In [6]:
# --- 4. Main Execution Block ---
if __name__ == "__main__":
    if not os.getenv("GROQ_API_KEY"):
        print("❌ ERROR: GROQ_API_KEY not found! Please ensure it is in your .env file.")
    else:
        # Define the list of tools the agent can use
        tools = [get_article_summary, generate_headlines]

        # Get the prompt template for the agent
        prompt = hub.pull("hwchase17/react")

        # Create the agent by combining the LLM, tools, and prompt
        agent = create_react_agent(llm, tools, prompt)

        # Create the Agent Executor, which runs the agent's logic
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

        print("🤖 Agent is ready. Ask me to generate headlines for a URL.")
        print("   Example: Get headlines for https://www.your-news-article-url.com")
        print("   Type 'exit' to quit.\n")

        # Start a loop to chat with the agent
        while True:
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                print("🤖 Goodbye!")
                break
            
            try:
                # Invoke the agent executor with the user's input
                result = agent_executor.invoke({"input": user_input})
                
                # Print the agent's final answer
                print("\n✅ Agent's Final Answer:")
                print(result['output'])
                print("-" * 50)
            
            except Exception as e:
                print(f"\n❌ An error occurred: {e}")

C:\Users\hp\anaconda\envs\Chameleon\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


🤖 Agent is ready. Ask me to generate headlines for a URL.
   Example: Get headlines for https://www.your-news-article-url.com
   Type 'exit' to quit.



You:  https://edition.cnn.com/2025/07/21/politics/trump-netanyahu-israel-gaza-syria




> Entering new AgentExecutor chain...
To answer the question related to the given URL, I first need to understand the content of the article. The URL appears to be a news article from CNN about a political situation involving Trump, Netanyahu, Israel, Gaza, and Syria.

Thought: I should start by getting a summary of the article to understand its content.
Action: get_article_summary
Action Input: https://edition.cnn.com/2025/07/21/politics/trump-netanyahu-israel-gaza-syria
➡️ [Tool Running: get_article_summary] Loading content from https://edition.cnn.com/2025/07/21/politics/trump-netanyahu-israel-gaza-syria...
➡️ [Tool Running: get_article_summary] Splitting and summarizing...


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


...



> Entering new AgentExecutor chain...
Thought: I need to understand the content of the article at the given URL to answer questions about it. The best action is to get a summary of the article.

Action: get_article_summary
Action Input: https://timesofindia.indiatimes.com/india/people-who-are-saying-shashi-tharoor-hits-back-at-kerala-congress-snub-responds-to-no-longer-one-of-us-remark/articleshow/122832291.cms
➡️ [Tool Running: get_article_summary] Loading content from https://timesofindia.indiatimes.com/india/people-who-are-saying-shashi-tharoor-hits-back-at-kerala-congress-snub-responds-to-no-longer-one-of-us-remark/articleshow/122832291.cms...
➡️ [Tool Running: get_article_summary] Splitting and summarizing...
➡️ [Tool Running: get_article_summary] Summary complete.
There are four separate summaries provided, each covering different topics. Here is a concise summary of each:

**Summary 1: Shashi Tharoor Controversy**

Shashi Tharoor, a Congress MP, faced backlash from Kerala Cong

You:  https://www.hindustantimes.com/india-news/air-india-completes-boeing-inspections-after-deadly-ahmedabad-crash-101753171132341.html




> Entering new AgentExecutor chain...
Thought: I need to understand the content of the article at the given URL to answer any questions related to it. The best course of action is to summarize the article.

Action: get_article_summary
Action Input: https://www.hindustantimes.com/india-news/air-india-completes-boeing-inspections-after-deadly-ahmedabad-crash-101753171132341.html
➡️ [Tool Running: get_article_summary] Loading content from https://www.hindustantimes.com/india-news/air-india-completes-boeing-inspections-after-deadly-ahmedabad-crash-101753171132341.html...
➡️ [Tool Running: get_article_summary] Splitting and summarizing...
➡️ [Tool Running: get_article_summary] Summary complete.
The article discusses Air India completing Boeing inspections following a fatal crash in Ahmedabad, but access is restricted.It seems like the observation got cut off. Let's continue with what we have.

Thought: Since the observation was incomplete, let's assume we didn't get the full summary. Howe

You:  exit


🤖 Goodbye!
